In [54]:
%%time
#Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pandas.plotting import autocorrelation_plot
import pmdarima as pm
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import os

os.getcwd()

Wall time: 0 ns


'C:\\Users\\marin\\Documents\\Controlador PID\\Bases'

## Abrir la BM y filtrar sólo por salidas a partir de enero del 2018

In [55]:
%%time
#Tomar datos que son desde el 2018
BM  = pd.read_csv('BM.csv')
lista = pd.read_csv('articulos_experimento.csv')
lista = lista["0"].to_numpy()


BM["Fecha"]=pd.to_datetime(BM.Fecha)
BM = BM[BM["Fecha"]>="2018-01-01"]
BM['Semana'] = pd.to_datetime(BM.Fecha.dt.week.astype(str)+BM.Fecha.dt.year.astype(str).add('-1') ,format='%V%G-%u')
BM['Cantidad'] = pd.to_numeric(BM['Cantidad'], errors='coerce')
BM["Entrada_Salida"]  = pd.to_numeric(BM["Entrada_Salida"], errors="coerce")
BM= BM[BM["Entrada_Salida"]==1]

Wall time: 5.98 s


In [56]:
%%time

id_articulos = BM['IdArticulo'].unique()
BM = BM[['IdArticulo','Fecha','Cantidad']]

# Set same time span for all articles
earliest_date = BM['Fecha'].min()
latest_date = BM['Fecha'].max()
temp_df = pd.DataFrame()

for articulo in id_articulos:
    temp_df = pd.concat([temp_df, pd.DataFrame({'IdArticulo':[articulo,articulo],
                                                'Fecha':[earliest_date,latest_date],
                                                'Cantidad':[0,0]})])
    
BM = pd.concat([BM, temp_df]).reset_index(drop='True');

Wall time: 2.95 s


In [57]:
%%time
data_weekly_grouped = BM.reset_index() \
                          .groupby('IdArticulo') \
                          .resample('W-MON', on='Fecha', label='left',closed='left')
data_weekly = pd.DataFrame(data_weekly_grouped.sum()['Cantidad']).reset_index().set_index('IdArticulo')
data_weekly = data_weekly.reset_index()
data_weekly["Semana"]=data_weekly["Fecha"]

BM = data_weekly

Wall time: 16.7 s


In [58]:
BM = BM.replace({0.0:0.000001})
BM

,IdArticulo,Fecha,Cantidad,Semana
0,501.0,2018-01-01,0.000001,2018-01-01
1,501.0,2018-01-08,0.000001,2018-01-08
2,501.0,2018-01-15,0.000001,2018-01-15
3,501.0,2018-01-22,0.000001,2018-01-22
4,501.0,2018-01-29,0.000001,2018-01-29
...,...,...,...,...
610199,4300003.0,2020-09-28,0.000001,2020-09-28
610200,4300003.0,2020-10-05,0.000001,2020-10-05
610201,4300003.0,2020-10-12,0.000001,2020-10-12
610202,4300003.0,2020-10-19,0.000001,2020-10-19


### Contar el número de semanas que han transcurrido desde Enero del 2018

In [59]:
%%time
semanas=len(BM.groupby("Semana").count())
print("Han Transcurrido:",semanas,"Semanas desde Enero del 2018")

Han Transcurrido: 148 Semanas desde Enero del 2018
Wall time: 18.5 ms


## Lista de los artículos que se encuentran en la base

In [60]:
%%time

prueba= lista[:2]
prueba

Wall time: 0 ns


array([700008., 700012.])

In [63]:
%%time
#Loop para hacer predicciones con los filtros

IdArticulo  =[]
SES3        =[]
SES5        =[]
SESA        =[]

for i in lista:
    print(i)

    #Crear la Serie de tiempo para un articulo en específico
    Medicamento = BM[BM["IdArticulo"]==i]
    SerieTiempo = Medicamento[["Semana", "Cantidad"]].groupby("Semana").sum();
    
    serie=SerieTiempo["Cantidad"]
    fit1 = SimpleExpSmoothing(serie).fit(smoothing_level=0.3,optimized=False)
    fcast1 = fit1.forecast(1).rename(r'$\alpha=0.3$')
    fit2 = SimpleExpSmoothing(serie).fit(smoothing_level=0.5,optimized=False)
    fcast2 = fit2.forecast(1).rename(r'$\alpha=0.5$')
    fit3 = SimpleExpSmoothing(serie).fit()
    fcast3 = fit3.forecast(1).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    
    
    IdArticulo.append(i)
    SES3.append(round(fcast1[0]))
    SES5.append(round(fcast2[0]))
    SESA.append(round(fcast3[0]))
    
d={"IdArticulo":IdArticulo, "SES3":SES3, "SES5":SES5,"SESA":SESA}
Predicciones_Smoothing=pd.DataFrame(data=d)
Predicciones_Smoothing

700008.0
700012.0
700024.0
700043.0
700046.0
700052.0
700072.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning: invalid value encountered in greater_equal
  loc = initial_p >= ub
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-

700077.0
700080.0
800017.0
800031.0
800050.0
800051.0
800093.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarn

800103.0
800116.0
800122.0
800148.0
800164.0
800182.0
800296.0
800297.0


...

3800025.0
3800035.0
3800041.0
3800051.0
3800064.0
3800065.0
3800066.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarn

3800067.0
3800068.0
3800069.0
3800075.0
Wall time: 6.35 s


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarn

,IdArticulo,SES3,SES5,SESA
0,700008.0,25.0,15.0,33.0
1,700012.0,2.0,1.0,4.0
2,700024.0,36.0,31.0,36.0
3,700043.0,36.0,32.0,37.0
4,700046.0,3.0,3.0,1.0
...,...,...,...,...
224,3800066.0,150.0,152.0,150.0
225,3800067.0,1416.0,1413.0,1412.0
226,3800068.0,1040.0,1021.0,1028.0
227,3800069.0,535.0,521.0,524.0


In [64]:
Predicciones_Smoothing.to_csv("Predicciones_S.csv", index=False)

In [65]:
%%time
#Loop para hacer predicciones con los filtros

IdArticulo  =[]
H3        =[]
H5        =[]
HA        =[]

for i in lista:
    print(i)

    #Crear la Serie de tiempo para un articulo en específico
    Medicamento = BM[BM["IdArticulo"]==i]
    SerieTiempo = Medicamento[["Semana", "Cantidad"]].groupby("Semana").sum();
    
    serie=SerieTiempo["Cantidad"]
    fit1 = Holt(serie).fit(smoothing_level=0.3)
    fcast4 = fit1.forecast(1).rename("Holt's linear trend")
    fit2 = Holt(serie).fit(smoothing_level=0.5)
    fcast5 = fit2.forecast(1).rename("Exponential trend")
    fit3 = Holt(serie).fit()
    fcast6 = fit3.forecast(1).rename("Additive damped trend")
    
    
    IdArticulo.append(i)
    H3.append(round(fcast4[0]))
    H5.append(round(fcast5[0]))
    HA.append(round(fcast6[0]))
    
d={"IdArticulo":IdArticulo, "H3":H3,"H5":H5, "HA": HA}
Prediccioones_Holt=pd.DataFrame(data=d)
Prediccioones_Holt

700008.0
700012.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning: invalid value encountered in greater_equal
  loc = initial_p >= ub
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-

700024.0
700043.0
700046.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)


700052.0
700072.0


...

3800066.0
3800067.0


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)


3800068.0
3800069.0
3800075.0
Wall time: 9.34 s


C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
C:\Users\marin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarn

,IdArticulo,H3,H5,HA
0,700008.0,25.0,15.0,41.0
1,700012.0,2.0,1.0,3.0
2,700024.0,37.0,31.0,38.0
3,700043.0,37.0,33.0,41.0
4,700046.0,3.0,3.0,2.0
...,...,...,...,...
224,3800066.0,151.0,153.0,151.0
225,3800067.0,1433.0,1424.0,1423.0
226,3800068.0,1040.0,1021.0,1028.0
227,3800069.0,535.0,521.0,524.0


In [66]:
Prediccioones_Holt.to_csv("Predicciones_Holt.csv", index=False)